In [1]:
import requests
import pandas

In [2]:
API_KEY = "I1ZPGZT3TBFALW8H"
BASE_URL = "https://www.alphavantage.co/query"

symbol = ['TSLA', 'AAPL']
periods = 4

EarningsSurpriseWeight = (1/7)
SalesGrowthWeight = (1/7)
OperatingMarginWeight = (1/7)
EarningsGrowthWeight = (1/7)
EarningsMomentumWeight = (1/7)
ROEWeight = (1/7)
OperatingCashFlowGrowthWeight = (1/7)



In [3]:
def fetch_earnings_data(symbol):
    FUNCTION = 'EARNINGS'
    response = requests.get(BASE_URL, params={
        'function': FUNCTION,
        'symbol': symbol,
        'apikey': API_KEY
    })
    response.raise_for_status()  # Automatically handle errors
    return response.json().get('quarterlyEarnings', [])

def summarize_earnings_for_periods(symbol, periods):
    earnings_data = fetch_earnings_data(symbol)
    
    if not earnings_data:
        return "No earnings data available."
    
    # Limit the number of periods to the available data
    periods = min(periods, len(earnings_data))
    summaries = []

    for i in range(periods):
        period_data = earnings_data[i]
        summary = {
            "Fiscal Date": period_data['fiscalDateEnding'],
            "Reported EPS": float(period_data['reportedEPS']),
            "Estimated EPS": float(period_data['estimatedEPS']),
            "Surprise": float(period_data['surprise']),
            "Surprise (%)": float(period_data['surprisePercentage']),
        }
        summaries.append(summary)
    
    return summaries

In [4]:
def fetch_balance_sheet_data(symbol):
    FUNCTION = 'BALANCE_SHEET'
    response = requests.get(BASE_URL, params={
        'function': FUNCTION,
        'symbol': symbol,
        'apikey': API_KEY
    })
    response.raise_for_status()  # Automatically handle errors
    return response.json().get('annualReports', [])

def summarize_balance_sheet_for_periods(symbol, periods):
    balance_sheet_data = fetch_balance_sheet_data(symbol)
    
    if not balance_sheet_data:
        return "No balance sheet data available."
    
    # Limit the number of periods to the available data
    periods = min(periods, len(balance_sheet_data))
    summaries = []

    for i in range(periods):
        period_data = balance_sheet_data[i]
        summary = {
            "Fiscal Date": period_data['fiscalDateEnding'],
            "Shareholders Equity": float(period_data['totalShareholderEquity']),
        }
        summaries.append(summary)
    
    return summaries

In [5]:
def fetch_income_statement_data(symbol):
    FUNCTION = 'INCOME_STATEMENT'
    response = requests.get(BASE_URL, params={
        'function': FUNCTION,
        'symbol': symbol,
        'apikey': API_KEY
    })
    response.raise_for_status()  # Automatically handle errors
    return response.json().get('annualReports', [])

def summarize_income_statement_for_periods(symbol, periods):
    income_data = fetch_income_statement_data(symbol)
    
    if not income_data:
        return "No income statement data available."
    
    # Limit the number of periods to the available data
    periods = min(periods, len(income_data))
    summaries = []

    for i in range(periods):
        period_data = income_data[i]
        summary = {
            "Fiscal Date": period_data['fiscalDateEnding'],
            "Revenue": float(period_data['totalRevenue']),
            "Operating Income": float(period_data['operatingIncome']),
            "Net Income": float(period_data['netIncome']),
        }
        summaries.append(summary)
    
    return summaries

In [6]:
def fetch_cash_flow_data(symbol):
    FUNCTION = 'Cash_Flow'
    response = requests.get(BASE_URL, params={
        'function': FUNCTION,
        'symbol': symbol,
        'apikey': API_KEY
    })
    response.raise_for_status()  # Automatically handle errors
    return response.json().get('annualReports', [])

def summarize_cash_flow_for_periods(symbol, periods):
    cash_flow_data = fetch_cash_flow_data(symbol)
    
    if not cash_flow_data:
        return "No cash flow data available."
    
    # Limit the number of periods to the available data
    periods = min(periods, len(cash_flow_data))
    summaries = []

    for i in range(periods):
        period_data = cash_flow_data[i]
        summary = {
            "Fiscal Date": period_data['fiscalDateEnding'],
            "Operating Cash Flow": float(period_data['operatingCashflow']),
        }
        summaries.append(summary)
    
    return summaries


In [7]:
# These are all the functions that do the math

def Positive_Earnings_Surprises(expectedEPS, reportedEPS):
    # Earnings Surprise (%)
    return ((reportedEPS - expectedEPS) / expectedEPS) * 100

def Increasing_Sales_Growth(currentPeriodSales, previousPeriodSales):
    # Sales Growth (%)
    return ((currentPeriodSales - previousPeriodSales) / previousPeriodSales) * 100

def Expanding_Operating_Margins(operatingIncome, netSales):
    # Operating Margin (%)
    return (operatingIncome / netSales) * 100

def Strong_Cash_Flow(operatingCashFlow):
    # Strong Cash Flow is subjective; can be a threshold check or percentage of net income
    return operatingCashFlow

def Earnings_Growth(currentPeriodNetIncome, pastPeriodNetIncome):
    # Earnings Growth (%)
    return ((currentPeriodNetIncome - pastPeriodNetIncome) / pastPeriodNetIncome) * 100

def Positive_Earnings_Momentum(recentEPSGrowthRate, pastEPSGrowthRate):
    # Earnings Momentum (%)
    return ((recentEPSGrowthRate - pastEPSGrowthRate) / pastEPSGrowthRate) * 100

def High_Return_on_Equity(netIncome, shareholdersEquity):
    # Return on Equity (%)
    return (netIncome / shareholdersEquity) * 100

In [8]:
def calc_earnings_surprise_percent():
    #expectedEPS = real_earnings_dict[1]['Estimated EPS']
    expectedEPS = earnings_summaries[0]['Estimated EPS']
    reportedEPS = earnings_summaries[0]['Reported EPS']
    result = Positive_Earnings_Surprises(expectedEPS, reportedEPS)
    return result

def calc_sales_growth():
    current_period_sales = income_summaries[0]['Revenue']
    previous_period_sales = income_summaries[1]['Revenue']
    result = Increasing_Sales_Growth(current_period_sales, previous_period_sales)
    return result

def calc_operating_margin():
    operating_income = income_summaries[0]['Operating Income']
    net_sales = income_summaries[0]['Revenue']
    result = Expanding_Operating_Margins(operating_income, net_sales)
    return result

def calc_earnings_growth():
    current_period_net_income = income_summaries[0]['Net Income']
    past_period_net_income = income_summaries[1]['Net Income']
    result = Earnings_Growth(current_period_net_income, past_period_net_income)
    return result

def calc_earnings_momentum():
    recent_eps_growth_rate = earnings_summaries[0]['Reported EPS'] - earnings_summaries[1]['Reported EPS']
    past_eps_growth_rate = earnings_summaries[1]['Reported EPS'] - earnings_summaries[2]['Reported EPS']
    result = Positive_Earnings_Momentum(recent_eps_growth_rate, past_eps_growth_rate)
    return result

def calc_roe():
    shareholders_equity = balance_sheet_summaries[0]['Shareholders Equity']
    current_period_net_income = income_summaries[0]['Net Income']
    result = High_Return_on_Equity(current_period_net_income, shareholders_equity)
    return result

def calc_operating_cash_flow_growth():
    recent_cash_flow_growth_rate = cash_flow_summaries[0]['Operating Cash Flow'] - cash_flow_summaries[1]['Operating Cash Flow']
    past_cash_flow_growth_rate = cash_flow_summaries[1]['Operating Cash Flow'] - cash_flow_summaries[2]['Operating Cash Flow']
    result = (recent_cash_flow_growth_rate / cash_flow_summaries[1]['Operating Cash Flow']) * 100
    return result

def calc_scores():
    
    esp_score = calc_earnings_surprise_percent() * EarningsSurpriseWeight
    sg_score = calc_sales_growth() * SalesGrowthWeight
    om_score = calc_operating_margin() * OperatingMarginWeight
    eg_score = calc_earnings_growth() * EarningsGrowthWeight
    em_score = calc_earnings_momentum() * EarningsMomentumWeight
    roe_score = calc_roe() * ROEWeight
    ocfg_score = calc_operating_cash_flow_growth() * OperatingCashFlowGrowthWeight
    total_score = esp_score + sg_score + om_score + eg_score + em_score + roe_score + ocfg_score
    results = [esp_score, sg_score, om_score, eg_score, em_score, roe_score, ocfg_score, total_score]
    return results

In [11]:
# Still need to implement multiple stocks in one results file
# Need to look into ExcelWriter for that, should just be a loop otherwise

# writer = pandas.ExcelWriter('results.xlsx')
categories = ['Earnings Surprise Growth % Score =', 'Sales Growth Score =', 'Operating Margin Score =','Earnings Growth Score =','Earnings Momentum Score =','ROE Score =','Operating Cash Flow Growth Score =', 'Total Score =']
i = 0

temp = pandas.DataFrame({'Each stock requested has been analyzed, and has been placed in an excel sheet with the appropriate name.'})
temp.to_excel('results.xlsx', sheet_name = 'Information', index = False)

while(i<len(symbol)):
    
    try:  
        earnings_summaries = summarize_earnings_for_periods(symbol[i], periods)
        income_summaries = summarize_income_statement_for_periods(symbol[i], periods)
        balance_sheet_summaries = summarize_balance_sheet_for_periods(symbol[i], periods)
        cash_flow_summaries = summarize_cash_flow_for_periods(symbol[i], periods)
    except Exception as e:
        print(str(e)) 
    
    
    scores = calc_scores()
    df = pandas.DataFrame({'Category' : categories, 'Scores' : scores})
    with pandas.ExcelWriter('results.xlsx', mode = 'a', if_sheet_exists = 'replace') as writer:
        df.to_excel(excel_writer = writer, sheet_name = symbol[i], index = False)
    
    i = i+1
    
print("Done")

Done
